# Double bonding curve liquidity pool as automatic swap market maker

In [249]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import ssl
import time
import math
import random
import decimal


#disable ssl for cryptory API & virtualenv
ssl._create_default_https_context = ssl._create_unverified_context
np.set_printoptions(suppress=True)

Request Data from Compound.finance API. We request a time step every hour over the last 9 months. The API does not let you get more granular data per request so we do it in 3 loops.

In [283]:
secondsInMonth = 60 * 60 * 24 *30
cDaiAddress = '0xf5dce57282a584d2746faf1593d3121fcac444dc'
endTimestamp = math.floor(time.time())
    
startTimeStamp = endTimestamp - 3 * 30 * 24 * 60 * 60 #use the most recent 3 month's worth of data.
num_buckets = 3 * 24 * 30 #interest rate every hour

interestOverTime = pd.DataFrame()

for i in range(2,-1,-1): # we can pull up to 3 months of data at a time if taking data points every hour.
    endTimestamp = math.floor(time.time()) - i * secondsInMonth * 3
    startTimeStamp = endTimestamp - 3 * 30 * 24 * 60 * 60 #use three months before the start
    print("itteration %s start: %s. end: %s. num_buckets: %s" %(i, startTimeStamp, endTimestamp, num_buckets))
    requestURL = "https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=%s"% (cDaiAddress, startTimeStamp, endTimestamp, num_buckets)
    response = requests.get(requestURL)
#     print(response.json())
    outputFrame = pd.DataFrame.from_dict(response.json()['supply_rates'])
    interestOverTime = interestOverTime.append(outputFrame, ignore_index = True)
interestOverTime.head()

itteration 2 start: 1549880290. end: 1557656290. num_buckets: 2160
itteration 1 start: 1557656291. end: 1565432291. num_buckets: 2160
itteration 0 start: 1565432294. end: 1573208294. num_buckets: 2160


,block_number,block_timestamp,rate
0,7710690,1555435090,0.000000
1,7710930,1555438690,0.000000
2,7711650,1555449490,0.000000
3,7715010,1555499890,0.001255
4,7715970,1555514290,0.002507


Convert Data into dataframe

In [284]:
interestOverTime['block_time'] = pd.to_datetime(interestOverTime['block_timestamp'],unit='s')
interestOverTime['rate_per_block'] = interestOverTime['rate'] / (4 * 60 * 24 * 365)
interestOverTime['rate_per_hour'] = interestOverTime['rate'] / (24 * 365)
interestOverTime.tail()

,block_number,block_timestamp,rate,block_time,rate_per_block,rate_per_hour
4546,8894369,1573190294,0.042333,2019-11-08 05:18:14,2.013543e-08,0.000005
4547,8894609,1573193894,0.042369,2019-11-08 06:18:14,2.015265e-08,0.000005
4548,8894849,1573197494,0.042514,2019-11-08 07:18:14,2.022183e-08,0.000005
4549,8895089,1573201094,0.043155,2019-11-08 08:18:14,2.052657e-08,0.000005
4550,8895329,1573204694,0.043100,2019-11-08 09:18:14,2.050049e-08,0.000005


Plot Intrest rate over time

In [286]:
data = [go.Scatter(
        x=interestOverTime['block_time'],
        y=interestOverTime['rate']
    )]

layout = go.Layout(
    title='Compound Interst Rate Over time',
    yaxis=dict(title='Lending Rate (%)'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)
figure.show()

Define some helper functions.

In [288]:
#Defines the fixed rate offered by the pool as a function of utilization. See Notion for explinaiton
def poolFixedRate(poolUtilization, currentFloatingRate, isLong):
    alpha = 1
    if isLong:
        return currentFloatingRate * (1 + poolUtilization/alpha)
    if not isLong:
        return currentFloatingRate * (1 - poolUtilization/alpha)

In [28]:
#Future value NACC(nominal anual, compounding continiously). Assumes t1 and t2 are in seconds
def futureValueNACC(nominal, rate, t1, t2):
    return nominal * math.exp(rate * (t2 - t1) / (60 * 60 * 24 *365))

In [240]:
#Future value NACH(nominal anual, compounding hourly) Assumes t1 and t2 are in seconds
def futureValueNACH(nominal, rate, t1, t2):
    return nominal * (1 + rate/(24*365)) ** (((t2 - t1) * 24 * 365) / (60 * 60 * 24 * 365))

In [273]:
interestOverTime.head()

,block_number,block_timestamp,rate,block_time,rate_per_block,rate_per_hour
0,7858770,1557656127,0.1,2019-05-12 10:15:27,7.105966e-08,0.000011
1,7859010,1557659727,0.1,2019-05-12 11:15:27,7.028429e-08,0.000011
2,7859250,1557663327,0.1,2019-05-12 12:15:27,7.013885e-08,0.000011
3,7859490,1557666927,0.1,2019-05-12 13:15:27,7.000989e-08,0.000011
4,7859730,1557670527,0.1,2019-05-12 14:15:27,7.001463e-08,0.000011


In [274]:
interestOverTime.tail()

,block_number,block_timestamp,rate,block_time,rate_per_block,rate_per_hour
4305,8894369,1573190129,0.1,2019-11-08 05:15:29,2.013543e-08,0.000011
4306,8894609,1573193729,0.1,2019-11-08 06:15:29,2.015265e-08,0.000011
4307,8894849,1573197329,0.1,2019-11-08 07:15:29,2.022183e-08,0.000011
4308,8895089,1573200929,0.1,2019-11-08 08:15:29,2.052657e-08,0.000011
4309,8895329,1573204529,0.1,2019-11-08 09:15:29,2.050049e-08,0.000011


In [147]:
interestOverTime.loc[4 + 1,'block_timestamp']

1555568403

In [156]:
#Finds the index in the pandas frame that has the closest starting index to the time specified
def findClosestIndex(t):
    for i in range(0,len(interestOverTime['block_timestamp'])):
        if t >= interestOverTime.loc[i,'block_timestamp'] and t <= interestOverTime.loc[i + 1,'block_timestamp']:
            return(i)
    return(-1)

In [289]:
#Floating Value accrued between two time periods in the compound data set.
def floatingValue(nominal, t1, t2):
    assert t1 < t2, "invalid input time"
    assert t1 >= int(interestOverTime.head(1)['block_timestamp']), "t1 before time series start"
    assert t2 <= int(interestOverTime.tail(1)['block_timestamp']), "t2 after time series end"
    startingIndex = findClosestIndex(t1)
    endingIndex = findClosestIndex(t2)
    positionValue = nominal
    for i in range(startingIndex, endingIndex + 2):
        ratePerHour = interestOverTime.loc[i,'rate']/(24 * 365)
        positionValue = positionValue * (1 + ratePerHour)
    return positionValue

In [290]:
floatingValue(100,1557656127, 1573201210)

104.73687632245203

In [291]:
int(interestOverTime.head(1)['block_timestamp'])

1555435090

In [292]:
interestOverTime["rate"].mean()

0.09384849992777904

In [293]:
futureValueNACH(100,0.1,1557656127, 1573201210)

105.05279564622636

In [294]:
(1573201210-1555500003)/(60*24)

12292.50486111111

In [295]:
105.0431080126262608725661952/105.05279564622636

0.9999077831909137